In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

from lion_pytorch import Lion

In [3]:
# one_hot_allNaNs_agep: alpha = 0.122
# one_hot_allNaNs_schl: alpha = 0.123
# one_hot_pNaNs_agep: alpha = 0.125
# one_hot_pNaNs_schl: alpha = 0.125

In [3]:
# test run 
alpha = .123
data_set = 'one_hot_allNaNs_agep_small.csv'
intermediate_models = 'best_model_small.pth'
output_model = 'model_small.pth'

In [5]:
# run 1
#data_set = 'one_hot_allNaNs_agep.csv'
#intermediate_models = 'best_model_allNaNs_agep.pth'
#output_model = 'model_allNaNs_agep.pth'
#alpha = .122

In [6]:
# run 2
#data_set = 'one_hot_allNaNs_schl.csv'
#intermediate_models = 'best_model_allNaNs_schl.pth'
#output_model = 'model_allNaNs_schl.pth'
#alpha = 0.123

In [7]:
# run 3
# data_set = 'one_hot_pNaNs_agep.csv'
# intermediate_models = 'best_model_pNaNs_agep.pth'
# output_model = 'model_pNaNs_agep.pth'
# alpha = 0.125

In [8]:
# run 4
# data_set = 'one_hot_pNaNs_schl.csv'
# intermediate_models = 'best_model_pNaNs_schl.pth'
# output_model = 'model_pNaNs_schl.pth'
# alpha = 0.125

In [15]:
from vae import VAE
from vae import GroupSoftmax
from trainer import Trainer

In [5]:
# Device config 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [8]:
# input size not updated
group_sizes = [2, 5, 5, 11, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
test = VAE(433, 1500, 6, 500, group_sizes)

total_params = sum(p.numel() for p in test.parameters())
trainable_params = sum(p.numel() for p in test.parameters() if p.requires_grad)
print(trainable_params)


57670299


In [9]:
# Dataset import 
# Windows path
# data = pd.read_csv('A:/csv_hus/psam_hus_pus_filtered.csv')
# WSL path
data = pd.read_csv(f'/workspace/data/{data_set}')
data.head()

,TEN:owned or mortgaged,TEN:rented,TEN:nan,HHL:english,HHL:spanish,HHL:other indo-european,HHL:asian and pacific island languages,HHL:other,HHL:nan,VEH:no vehicles,...,SCHL_13:bachelor,SCHL_13:graduate,SCHL_13:nan,SCHL_14:other,SCHL_14:less than high school,SCHL_14:high school,SCHL_14:college or associate,SCHL_14:bachelor,SCHL_14:graduate,SCHL_14:nan
0,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [10]:
cols = list(data.columns)
cols = [col.split(":")[0] for col in cols]

onehot_counts = {col: sum(data.columns.str.startswith(f"{col}:")) for col in cols}
print(list(onehot_counts.values()))


input_dim = sum(onehot_counts.values())
print(input_dim)

[3, 6, 6, 12, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
439


In [11]:
# Data loader 

data_tensor = torch.tensor(data.values, dtype=torch.float32)
data_loader = DataLoader(data_tensor, batch_size=len(data_tensor), shuffle=True)

# Class_sizes 
group_sizes = list(onehot_counts.values())


In [25]:
model = VAE(input_dim, 1000, 6, 500, group_sizes)
# load pretrained model 
#params = torch.load('/workspace/models/model.pth')
#
#model.load_state_dict(params)

In [26]:
# optimizer
optimizer = Lion(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, device)

In [27]:
# train the model 
model_path = f'/workspace/models/{intermediate_models}'
trainer.train(data_loader, 4000, model_path, gamma=2, alpha=alpha)

Epoch 0, Loss: 1.6820757389068604, LR: 0.001
Epoch 1, Loss: 0.921448826789856, LR: 0.001
Epoch 2, Loss: 16.124601364135742, LR: 0.001
Epoch 3, Loss: 0.6596017479896545, LR: 0.001
Epoch 4, Loss: 0.6420459747314453, LR: 0.001
Epoch 5, Loss: 0.6318889856338501, LR: 0.001
Epoch 6, Loss: 0.6251727342605591, LR: 0.001
Epoch 7, Loss: 0.6199595928192139, LR: 0.001
Epoch 8, Loss: 0.6168729066848755, LR: 0.001
Epoch 9, Loss: 0.6129215955734253, LR: 0.001
Epoch 10, Loss: 0.6092024445533752, LR: 0.001
Epoch 11, Loss: 0.606913149356842, LR: 0.001
Epoch 12, Loss: 0.604936957359314, LR: 0.001
Epoch 13, Loss: 0.6009527444839478, LR: 0.001
Epoch 14, Loss: 0.6021181344985962, LR: 0.001
Epoch 15, Loss: 0.5984448790550232, LR: 0.001
Epoch 16, Loss: 0.5968568325042725, LR: 0.001
Epoch 17, Loss: 0.5946587324142456, LR: 0.001
Epoch 18, Loss: 0.592410147190094, LR: 0.001
Epoch 19, Loss: 0.5918180346488953, LR: 0.001
Epoch 20, Loss: 0.5899744033813477, LR: 0.001
Epoch 21, Loss: 0.5884053707122803, LR: 0.001
Ep

In [28]:
torch.save(model.state_dict(), f'/workspace/models/{output_model}')